In [ ]:
import os 
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from math import factorial
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
import gc
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.compose import make_column_transformer
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Model, Sequential
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

from mlxtend.classifier import StackingCVClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

from mlxtend.classifier import StackingCVClassifier
import shap

from sklearn import model_selection
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC

In [ ]:
# Reading train and testing datasets
output = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv')
train_df = pd.read_csv('../input/tpsfeb2022-v2/train_cluster_target.csv')
test_df = pd.read_csv('../input/tpsfeb2022-v2/test_cluster_target.csv')

In [ ]:
PROBAS = True
FOLDS = 5
N_ESTIMATORS = 100  #00

In [ ]:
le = LabelEncoder()
train_df["target"] = le.fit_transform(train_df["target"])
target = le.fit_transform(train_df.target)
train_df=train_df.drop(['sample_weight'], axis = 1)
train_df.shape

In [ ]:
TARGET='target'
X = train_df.drop([TARGET], axis = 1)
y = train_df[TARGET]

print (f'X:{X.shape} y: {y.shape} \n')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 2021)
print (f'X_train:{X_train.shape} y_train: {y_train.shape}')
print (f'X_test:{X_test.shape} y_test: {y_test.shape}')

In [ ]:
taken_classifiers = ["RandomForest", "DecisionTree", "CatBoost", "LGBM", "ExtraTrees"]

In [ ]:
models_scores_results, models_names = list(), list() 

In [ ]:
# This function searches best stacking configuration
def best_stacking_search():
    cls_list = []
    best_auc = -1
    i=0

    best_cls_experiment = list()

    print(">>>> Training started <<<<")

    for cls_comb in range(2, len(taken_classifiers)+1):
        for subset in itertools.combinations(taken_classifiers, cls_comb):
            cls_list.append(subset)

    print(f"Total number of model combination: {len(cls_list)}")


    for cls_exp in cls_list:
        cls_labels = list(cls_exp)

        classifier_exp = []
        for ii in range(len(cls_labels)):
            label = taken_classifiers[ii]
            classifier = classifiers[label]
            classifier_exp.append(classifier)


        sclf = StackingCVClassifier(classifiers = classifier_exp,
                                    shuffle = False,
                                    use_probas = True,
                                    cv = FOLDS,
                                    meta_classifier = mlr,
                                    n_jobs = -1)

        scores = model_selection.cross_val_score(sclf, X_train, y_train, cv = FOLDS, scoring='accuracy')

        if scores.mean() > best_auc:
            best_cls_experiment = list(cls_exp)
        i += 1
        print(f"  {i} - Stacked combination - Acc {cls_exp}: {scores.mean():.5f}")
        
    return best_cls_experiment

In [ ]:
cl2 = RandomForestClassifier(n_estimators=100, min_samples_split=12, max_depth=26, min_samples_leaf=1,max_features=0.34966)
cl3 = GaussianNB()
cl4 = DecisionTreeClassifier(max_depth = 5)
cl5 = CatBoostClassifier( verbose = None, logging_level = 'Silent')
cl6 = LGBMClassifier()
cl7 = ExtraTreesClassifier(n_estimators=1111,n_jobs=-1,verbose=0,random_state=1)
cl8 = MLPClassifier()
classifiers = {
    "RandomForest": cl2,
    "DecisionTree": cl4,
    "CatBoost": cl5,
    "LGBM": cl6,
    "ExtraTrees": cl7,
}

In [ ]:
best_cls_experiment = ['RandomForest', 'ExtraTrees','CatBoost']
#best_cls_experiment = ['RandomForest']
print(f'The choosen models configuration: {best_cls_experiment}')

classifier_exp = []
for label in best_cls_experiment:
        classifier = classifiers[label]
        classifier_exp.append(classifier)

In [ ]:
mlr =LogisticRegression(C = 0.1)

In [ ]:
scl = StackingCVClassifier(classifiers= classifier_exp,
                            meta_classifier = mlr, # use meta-classifier
                            use_probas = PROBAS,   # use_probas = True/False
                            random_state = 2021)

scores = model_selection.cross_val_score(scl, X_train, y_train, cv = FOLDS, scoring='accuracy')
models_scores_results.append(scores)
models_names.append('scl')
print("Meta model (slc) - accuracy: %0.5f " % (scores.mean()))
scl.fit(X_train, y_train)

top_meta_model = scl
base_acc = scores.mean()

In [ ]:
# And we approaching to the final stage ... prediction ... 

test_preds = scl.predict(test_df)
#test_preds = test_preds.astype(int)
test_preds

In [ ]:
from scipy import stats
stats.describe(test_preds)

In [ ]:
train_df2 = pd.read_csv('../input/tpsfeb2022-v2/train_cluster_target.csv')
le2 = LabelEncoder()
train_df2["target"] = le2.fit(train_df2["target"])

In [ ]:
test_preds =le2.inverse_transform(test_preds )

In [ ]:
test_preds

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submission['target'] = test_preds
submission.to_csv('submission.csv', index=False)

References:

https://www.kaggle.com/abdulravoofshaik/clustering-extra-tree-lb-98-34

https://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier/#example-2-using-probabilities-as-meta-features

https://www.kaggle.com/hasanbasriakcay/tps-feb22-pseudo-labels-stacking-0-97-lb

https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard


